# Search Umbra SAR Archive

Searching Archive requires an API Token!
https://docs.canopy.umbra.space/docs/migrate-to-stac-api-v2

You can *browse* public data (a static STAC catalog), but not search it:
https://registry.opendata.aws/umbra-open-data/

In [1]:
import geopandas as gpd
import pystac_client
import os

In [2]:
# Search for acquisitions in AWS Open Data Catalog
# NOTE: different endpoint, but still need auth
stac_api_url = "https://api.canopy.umbra.space/archive/"
archive = pystac_client.Client.open(stac_api_url,
                                    headers={"authorization": f"Bearer {os.environ.get('UMBRA_API_TOKEN')}" }
)
archive

# Hack fix for broken API links (need to be https://)
# https://github.com/huskysar/umbra/issues/1
for link in archive.get_links():
    link.target = link.target.replace('http://','https://')

In [3]:
cql2filter = {
    "op": "=",
    "args": [
      {
        "property": "umbra:open-data-catalog"
      },
      True
    ]
  }

# NOTE: testing API request limits
#limit_results=10000 #APIError: upstream request timeout
#limit_results=5000 #APIError: upstream request timeout
limit_results=2000 #works
#limit_results=1000 #works

stac_search = archive.search(
    bbox='-124.84, 24.39, -66.88, 49.38',
    filter=cql2filter,
    #NOTE: seems this must be set to>= max_items, else 'Unauthorized'
    max_items=limit_results,
    limit=limit_results,
)

items = stac_search.item_collection()
gf = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')
len(gf)

1537

In [4]:
gf.iloc[0]

geometry                                    POLYGON Z ((-111.78960703665004 39.10127878084...
id                                                       2f81176a-f346-4cb8-96eb-e1adb751ef80
created                                                      2025-01-14T12:41:12.173474+00:00
updated                                                      2025-01-14T12:41:12.173478+00:00
platform                                                                             Umbra-09
end_datetime                                                 2025-01-14T05:21:38.680145+00:00
umbra:task_id                                            34e27ed9-f9a3-43f9-8ee8-815175905ebe
start_datetime                                                      2025-01-14T05:21:37+00:00
sar:product_type                                                                          GEC
sar:looks_azimuth                                                                           1
sar:polarizations                                           

In [5]:
i = items[-1]
i.assets

{'thumbnail': <Asset href=https://api.canopy.umbra.space/archive/thumbnail/dce419d0-fb28-41cc-9dae-5221aab83f3e>}

## Notes

* Currently 1537 acquisitions in public data archive
* Unfortunately only thumbnails in assets!

Hmmm,,, So this catalog is different from the static STAC catalog here (which lists all assets...)
https://radiantearth.github.io/stac-browser/#/external/s3.us-west-2.amazonaws.com/umbra-open-data-catalog/stac/2025/2025-01/2025-01-04/7f40db7e-8e11-487e-8fbb-3985338f542b/7f40db7e-8e11-487e-8fbb-3985338f542b.json

In [6]:
# Search for all acquisitions in WA State

aoi = gpd.read_file('https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson')
aoi.explore()

In [7]:
cql2filter = {
    "op": "=",
    "args": [
      {
        "property": "umbra:open-data-catalog"
      },
      True
    ]
  }

limit_results=2000

stac_search = archive.search(
    #filter=cql2filter,
    max_items=limit_results,
    limit=limit_results,
    intersects=aoi.geometry.iloc[0],
)

items = stac_search.item_collection()
gf = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')
len(gf)

89

In [8]:
gf.explore(column='umbra:open-data-catalog', cmap='plasma', popup=True)

In [9]:
items[0]

<Item id=9d4118bb-0a8c-40ce-9471-3974dcf5646e>

In [10]:
from IPython.display import Image
def show_thumbnail(row):
    item = items[row.name]
    print(item.id)
    display(Image(url=item.assets['thumbnail'].href))

# Thumbnails are public even for non-open data
# Hmmm, no longer seems to be the case, maybe need a signed URL
show_thumbnail(gf.iloc[0])
show_thumbnail(gf.iloc[30])

9d4118bb-0a8c-40ce-9471-3974dcf5646e


ab655330-5629-4b47-948d-73c570a9d181


In [11]:
# Search based on single point

# For example, is this bistatic pair public?...
# http://umbra-open-data-catalog.s3.amazonaws.com/sar-data/tasks/ad%20hoc/bistatic/tarbela_dam/2024-02-20-16-57-20.tif
# No, seems it's not in the catalog

aoi_as_dict = {
        "coordinates": [
          72.698,
          34.099
        ],
        "type": "Point"
      }

cql2filter = {
    "op": "=",
    "args": [
      {
        "property": "umbra:open-data-catalog"
      },
      True
    ]
  }

limit_results=2000

stac_search = archive.search(
    #filter=cql2filter,
    max_items=limit_results,
    limit=limit_results,
    intersects=aoi_as_dict,
)

items = stac_search.item_collection()
gf = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')
len(gf)

2

In [12]:
gf.iloc[0]

geometry                                    POLYGON Z ((72.73966216664009 34.1003607578395...
id                                                       f5c610ed-8880-49a0-880c-12b2296201e8
created                                                      2024-03-06T19:57:37.840927+00:00
updated                                                      2024-03-06T19:57:37.840932+00:00
platform                                                                             Umbra-04
end_datetime                                                 2024-03-06T16:11:33.839781+00:00
umbra:task_id                                            6badf1a5-c9bd-4345-b18c-4f336712fa78
start_datetime                                                      2024-03-06T16:11:28+00:00
sar:product_type                                                                          GEC
sar:looks_azimuth                                                                           2
sar:polarizations                                           

In [13]:
gf.iloc[1]

geometry                                    POLYGON Z ((72.67192610282478 34.1034037196364...
id                                                       d840b9cd-403b-482d-90c0-9d10ea37b44d
created                                                                                   NaN
updated                                                                                   NaN
platform                                                                             Umbra-05
end_datetime                                                        2023-10-04T04:28:40+00:00
umbra:task_id                                            0fbdabe8-c3cd-4a3c-a0ef-4cfaada9b0ad
start_datetime                                                      2023-10-04T04:28:32+00:00
sar:product_type                                                                          GEC
sar:looks_azimuth                                                                           1
sar:polarizations                                           